#### DATA 319, Spring 2024
#### Quiz 3
##### Based on: Stanford CS246 class - Colab 4 assignment

## Collaborative Filtering

In [1]:
from IPython.display import Image
print("Colab 4 Mascot")
Image(url='https://media4.giphy.com/media/wtdVYmaRWJ1PyPcc8e/giphy.gif?cid=ecf05e478jdbzo92b89f2b5ud5184xda9cen1xce4bwmjeyu&ep=v1_stickers_search&rid=giphy.gif&ct=s',width=150)

Colab 4 Mascot


### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=87cacaf6752190e7a86c2bdc30e4063cdcc7934a85eaa9ff4d18d425e2a7ac49
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [7]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [8]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [9]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [10]:
''' 3 lines of code in total expected.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
num_training = training.count()
num_test = test.count()
num_movies = items.count()

print(num_training)
print(num_test)
print(num_movies)


80000
20000
1682


Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [21]:
''' 5-6 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

als = ALS(maxIter=20, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating", coldStartStrategy="drop")

model = als.fit(training)

predictions = model.transform(test)


Now compute the RMSE on the test dataset.

In [22]:
''' 4-5 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

RMSE = evaluator.evaluate(predictions)

print(RMSE)

1.1623851126614138


At this point, you can use the trained model to produce the top-K recommendations for each user.

In [49]:
''' 9-10 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
K = 5

user_recs = model.recommendForAllUsers(K)
user_recs.show(truncate = False)

+-------+---------------------------------------------------------------------------------------------+
|user_id|recommendations                                                                              |
+-------+---------------------------------------------------------------------------------------------+
|1      |[{1192, 6.6268296}, {1368, 6.412632}, {1127, 6.297151}, {884, 6.2213182}, {1245, 6.074142}]  |
|2      |[{854, 7.4585276}, {1021, 7.3485518}, {915, 7.283166}, {536, 7.240408}, {279, 7.086167}]     |
|3      |[{1615, 8.967927}, {1131, 8.472828}, {844, 8.251671}, {634, 8.131035}, {951, 7.9794846}]     |
|4      |[{1205, 11.500933}, {394, 10.867035}, {1639, 9.18111}, {206, 9.104204}, {593, 9.028999}]     |
|5      |[{1199, 9.264165}, {1192, 8.924071}, {360, 8.48603}, {908, 7.8133307}, {1598, 7.251363}]     |
|6      |[{733, 6.2995844}, {1643, 6.133836}, {1311, 5.7547946}, {765, 5.554703}, {602, 5.542717}]    |
|7      |[{1311, 7.0023327}, {1664, 6.6614347}, {1643, 6.4968624

Once you have working code for the recommendations above, output the top 5 ratings for the first user.

In [50]:
first_user = training.select("user_id").distinct().orderBy('user_id').first()['user_id'] #Get first User ID
users = spark.createDataFrame([Row(user_id = first_user)]) #create DF to hold entry for first user
user_recs = model.recommendForUserSubset(users, K) #k is set to 5 so it will make 5 recs
user_recs.show(truncate = False) #show recs

+-------+-------------------------------------------------------------------------------------------+
|user_id|recommendations                                                                            |
+-------+-------------------------------------------------------------------------------------------+
|1      |[{1192, 6.6268296}, {1368, 6.412632}, {1127, 6.297151}, {884, 6.2213182}, {1245, 6.074142}]|
+-------+-------------------------------------------------------------------------------------------+

